In [1]:
# import packages needed for entire model

import tensorflow as tf
from keras import layers, regularizers, metrics
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import pandas as pd
from sklearn.model_selection import train_test_split, ShuffleSplit
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import h5py

In [ ]:
# Takes Random Data of a specified size
splicingVal = 11
numOutputs = int(220/splicingVal)

# import data file, splice data, split into testing and training data
file = "dataset_comp_image_spectra.h5"
hf = h5py.File(file, 'r')
imgs = hf['images'][()]
spectra = hf['spectra'][()]
atfrac = np.transpose(hf['atfrac'][()])
z = list(zip(imgs, spectra, atfrac))
badIndices = [i for i, each in enumerate(imgs) if np.sum(each[16:48][16:48]) > 2450 or np.sum(each) < 325 or round(np.sum(atfrac[i])) == 0]
badIndices.reverse()
for index in badIndices:
  del z[index]
imgs, spectra, atfrac = zip(*z)
imgs, spectra = list(imgs), list(spectra)
spectra = [x[::splicingVal] for x in spectra]
x_valid = np.array(imgs[:2000])
y_valid = np.array(spectra[:2000])
del atfrac, z, imgs[0:2000], spectra[0:2000]
imgs, spectra = np.array(imgs), np.array(spectra)

In [3]:
# Current Neural Network Model function

def get_conv_model_normal():

    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(units=numOutputs, activation = 'sigmoid')])
    model.summary()
    adamOpti = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(optimizer= adamOpti, loss='mse', metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

In [10]:
# Running Neural Network with averaging Ensemble, including tensorboard and early stopping

# Define number of networks, batches, and epochs 
epochs = 2
batch_size = 32
log_folder = 'logs'
# Run Neural Network for the desired number of networks in the ensemble (numNetworks)
data = ShuffleSplit(n_splits=1, random_state=0, test_size=0.20, train_size=None)
data.get_n_splits(imgs, spectra)
for train_index, test_index in data.split(imgs, spectra):
  normal_model = get_conv_model_normal()
  x_train, x_test = imgs[train_index], imgs[test_index]
  y_train, y_test = spectra[train_index], spectra[test_index]
  checkpoint_filepath = 'checkpoint.h5'
  early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath, save_weights_only=True, 
                monitor='val_mean_squared_error', mode='max', save_best_only=True)
  normal_history = normal_model.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, validation_data = (x_test, y_test), verbose = 1,
                callbacks= [early_stopping, model_checkpoint_callback])
  checkpoint = h5py.File(checkpoint_filepath, 'a')
  checkpoint.create_dataset('x_valid', data = x_valid, maxshape = np.shape(x_valid))
  checkpoint.create_dataset('y_valid', data = y_valid, maxshape = np.shape(y_valid))
normal_model.save('Model #.h5')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 dense_24 (Dense)            (None, 62, 62, 256)       16640     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 256)      0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 31, 31, 256)       0         
                                                                 
 flatten_6 (Flatten)         (None, 246016)            0         
                                                                 
 dense_25 (Dense)            (None, 256)               62980352  
                                                      

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2
198/198 [==============================] - 9s 45ms/step - loss: 0.0696 - mean_squared_error: 0.0696 - val_loss: 0.0659 - val_mean_squared_error: 0.0659
Epoch 2/2
198/198 [==============================] - 8s 40ms/step - loss: 0.0652 - mean_squared_error: 0.0652 - val_loss: 0.0659 - val_mean_squared_error: 0.0659
<KeysViewHDF5 ['conv2d_6', 'dense_24', 'dense_25', 'dense_26', 'dense_27', 'dropout_6', 'flatten_6', 'max_pooling2d_6', 'top_level_model_weights', 'x_valid', 'y_valid']>
